In [1]:
#필요 라이브러리 import

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
pio.renderers.default = "notebook_connected"
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
%matplotlib inline
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import plotly.express as px

In [2]:
def pie_chart(data, col, title = ''):
    cnt_df = data[col].value_counts().reset_index()
    fig = px.pie(cnt_df[:10], 
                 values = col, 
                 names = 'index', 
                 title = title, 
                 template = 'seaborn',
                 color_discrete_sequence=px.colors.sequential.RdBu)
    fig.update_traces(rotation=90, pull=0.05, textinfo="value+percent+label")
    fig.show()

In [6]:
df_seoul = pd.read_csv('../../01_covid19_analysis/data/서울시 코로나19 확진자 현황.csv', encoding='ms949')
df_seoul.head()

,연번,확진일,환자번호,국적,환자정보,지역,여행력,접촉력,조치사항,상태,이동경로,등록일,수정일,노출여부
0,4131,9.02.,20637,NaN,NaN,강서구,NaN,기타 확진자 접촉,NaN,NaN,확인 중,2020-09-03 11:11:12,2020-09-03 11:11:12,Y
1,4130,9.02.,20628,NaN,NaN,강동구,NaN,확인 중,NaN,NaN,확인 중,2020-09-03 11:11:12,2020-09-03 11:11:12,Y
2,4129,9.02.,20590,NaN,NaN,노원구,NaN,다래경매 관련,NaN,NaN,확인 중,2020-09-03 11:11:12,2020-09-03 11:11:12,Y
3,4128,9.02.,20614,NaN,NaN,서초구,NaN,기타 확진자 접촉,NaN,NaN,확인 중,2020-09-03 11:11:12,2020-09-03 11:11:12,Y
4,4127,9.02.,20596,NaN,NaN,노원구,NaN,성북구 사랑제일교회 관련,NaN,NaN,확인 중,2020-09-03 11:11:12,2020-09-03 11:11:12,Y


In [7]:
df_province = df_seoul.copy()
pie_chart(df_province, '지역',title = '지역별 확진자 비율')

## 성북구가 가장 많은 비율을 차지 하는데 이유는 ??

In [8]:
pie_chart(df_province, '접촉력',title = '집단전염 비율')

## 성북구 사랑제일교회 관련 확진자가 많은데.. 다른 교회는 얼마나 있을까

In [9]:
#반장님 코드
contact = df_province['접촉력'].value_counts()
church = []
for i in contact.index:
    if '교회' in i:
        church.append(i)
church_peoples = df_province[df_province['접촉력'].isin(church)]
church_peoples['접촉력'].value_counts()

pie_chart(church_peoples, '접촉력',title = ' ')